# Term and semantic matching

- [Part 1, Term-based matching]:
    - Learn how to load a dataset and process it.
    - Standard IR methods (TF-IDF, BM25, QL).
    - Evaluating IR methodss.
- [Part 2, Semantic-based matching]:
    - Vector-space retrieval methods (LSI, LDA).
    - Re-ranking using LSI and LDA
    

In [ ]:
# imports 

import os
import zipfile
from functools import partial

import nltk
import requests
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from ipywidgets import widgets
from IPython.display import display, HTML
#from IPython.html import widgets
from collections import namedtuple

%matplotlib inline


# Part 1: Term-based Matching 

[Back to top](#top)

The CACM dataset is a collection of titles and abstracts from the journal CACM (Communication of the ACM).

Table of contents:
- [Section 1: Text Processing](#text_processing) 
- [Section 2: Indexing](#indexing) 
- [Section 3: Ranking](#ranking) 
- [Section 4: Evaluation](#evaluation) 

---
## Section 1: Text Processing 

[Back to Part 1](#part1)


The following cell downloads the dataset and unzips it to a local directory.

In [2]:
def download_dataset():
    folder_path = os.environ.get("IR1_DATA_PATH")
    if not folder_path:
        folder_path = "./datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    file_location = os.path.join(folder_path, "cacm.zip")
    
    # download file if it doesn't exist
    if not os.path.exists(file_location):
        
        url = "https://surfdrive.surf.nl/files/index.php/s/M0FGJpX2p8wDwxR/download"

        with open(file_location, "wb") as handle:
            print(f"Downloading file from {url} to {file_location}")
            response = requests.get(url, stream=True)
            for data in tqdm(response.iter_content()):
                handle.write(data)
            print("Finished downloading file")
    
    if not os.path.exists(os.path.join(folder_path, "train.txt")):
        
        # unzip file
        with zipfile.ZipFile(file_location, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        
download_dataset()

In [ ]:
def read_cacm_docs(root_folder = "./datasets/"):
    """
        Reads in the CACM documents. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (doc_id, document), where 'document' is a single string created by 
            appending the title and abstract (separated by a "\n"). 
            In case the record doesn't have an abstract, the document is composed only by the title
    """
    # YOUR CODE HERE
    
    file_location = os.path.join(root_folder, "cacm.all")

    doclist = []

    with open(file_location, "r") as f:
        docs = [("I\n" + x) for x in f.read().split(".I ")]

    docs = docs[1:]
    docs_fields = [["." + x for x in doc.split("\n.")] for doc in docs]

    for doc in docs_fields:
        abstract = ""
        for field in doc:
            if field.startswith('.I\n') == True:
                doc_id = field[3:]
            elif field.startswith('.T\n') == True:
                title = field[3:]
            elif field.startswith('.W\n') == True:
                abstract = "\n" + field[3:]
        document = title + abstract
        doclist.append((int(doc_id), document))
    
    return doclist
    
    raise NotImplementedError()

In [ ]:

docs = read_cacm_docs()

assert isinstance(docs, list)
assert len(docs) == 3204, "There should be exactly 3024 documents"


In [8]:
def read_queries(root_folder = "./datasets/"):
    """
        Reads in the CACM queries. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (query_id, query)
    """
    
    file_location = os.path.join(root_folder, "query.text")

    querylist = []

    with open(file_location, "r") as f:
        queries = [("I\n" + x) for x in f.read().split(".I ")]

    queries = queries[1:]
    query_fields = [["." + x for x in query.split("\n.")] for query in queries]

    for query in query_fields:
        for field in query:
            if field.startswith('.I\n') == True:
                query_id = field[3:]
            elif field.startswith('.W\n') == True:
                query_w = field[3:]
        querylist.append((int(query_id), query_w))
    
    return querylist
    
    raise NotImplementedError()


In [11]:
# TODO: Implement this! (1 point)
def load_stopwords(root_folder = "./datasets/"):
    """
        Loads the stopwords. The dataset is assumed to be in the folder "./datasets/" by default
        Output: A set of stopwords
    """    
    file_location = os.path.join(root_folder, "common_words")

    with open(file_location, "r") as f:
        stopwords = {x for x in f.read().lower().split("\n") if x != ""}
    
    return stopwords

    raise NotImplementedError()


In [13]:
def tokenize(text):
    """
        Tokenizes the input text. Use the WordPunctTokenizer
        Input: text - a string
        Output: a list of tokens
    """
    
    tokens = nltk.tokenize.WordPunctTokenizer().tokenize(text)
    
    return tokens
    
    raise NotImplementedError()

In [14]:
text = "the quick brown fox jumps over the lazy dog"
tokens = tokenize(text)

assert isinstance(tokens, list)
assert len(tokens) == 9

print(tokens)

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


In [15]:
def stem_token(token):
    """
        Stems the given token using the PorterStemmer from the nltk library
        Input: a single token
        Output: the stem of the token
    """
    
    stem = nltk.stem.porter.PorterStemmer().stem(token)
    
    return stem
    
    raise NotImplementedError()

In [16]:
assert stem_token('owned') == 'own'
assert stem_token('itemization') == 'item'


In [17]:
def process_text(text, stem=False, remove_stopwords=False, lowercase_text=False):
    
    tokens = []
    for token in tokenize(text):
        if remove_stopwords and token.lower() in stopwords:
            continue
        if stem:
            token = stem_token(token)
        if lowercase_text:
            token = token.lower()
        tokens.append(token)

    return tokens 

In [18]:
# In this configuration:
# Don't preprocess the text, except to tokenize 
config_1 = {
  "stem": False,
  "remove_stopwords" : False,
  "lowercase_text": True
} 


# In this configuration:
# Preprocess the text, stem and remove stopwords
config_2 = {
  "stem": True,
  "remove_stopwords" : True,
  "lowercase_text": True, 
} 
doc_repr_1 = []
doc_repr_2 = []
for (doc_id, document) in docs:
    doc_repr_1.append((doc_id, process_text(document, **config_1)))
    doc_repr_2.append((doc_id, process_text(document, **config_2)))

--- 

## Section 2: Indexing

[Back to Part 1](#part1)



A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. We consider a simple inverted index, which maps a word to a set of documents.

In [19]:
def build_tf_index(documents):
    """
        Build an inverted index (with counts). The output is a dictionary which takes in a token
        and returns a list of (doc_id, count) where 'count' is the count of the 'token' in 'doc_id'
        Input: a list of documents - (doc_id, tokens) 
        Output: An inverted index. [token] -> [(doc_id, token_count)]
    """    
    from collections import defaultdict
    
    index = defaultdict(str)

    for (doc_id, tokens) in documents:
        unique_tokens = set(tokens)
        for word in unique_tokens:
            count_word = int(tokens.count(word))
            if word not in index.keys():
                index[word] = [(doc_id, count_word)]
            else:
                index[word].append((doc_id, count_word))
    
    return index

    raise NotImplementedError()

In [20]:
# Create the 2 indices
tf_index_1 = build_tf_index(doc_repr_1)
tf_index_2 = build_tf_index(doc_repr_2)

# This function returns the tf_index of the corresponding config
def get_index(index_set):
    assert index_set in {1, 2}
    return {
        1: tf_index_1,
        2: tf_index_2
    }[index_set]

# This function preprocesses the text given the index set, according to the specified config
def preprocess_query(text, index_set):
    assert index_set in {1, 2}
    if index_set == 1:
        return process_text(text, **config_1)
    elif index_set == 2:
        return process_text(text, **config_2)



---
## Section 3: Ranking 

[Back to Part 1](#part1)

After cleaning and processing the dataset, the following IR systems are to considered which score documents against queries:
- [Section 3.1: Bag of Words](#bow)
- [Section 3.2: TF-IDF](#tfidf)
- [Section 3.3: Query Likelihood Model](#qlm)
- [Section 3.4: BM25](#bm25)

--- 

### Section 3.1: Bag of Words <a class="anchor" id="bow"></a>

In [23]:
# TODO: Implement this! (10 points)
def bow_search(query, index_set):
    """
        Perform a search over all documents with the given query. 
        Note: You have to use the `get_index` function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    
    query_score = []
    
    for query_word in processed_query:
        for key, value in index.items():
            if query_word == key:
                bow_value = [(doc_id, 1) for doc_id, token_count in value]
                query_score.extend(bow_value)
    
    dict_query = dict()
    for (key, value) in query_score:
        if key in dict_query:
            dict_query[key] += value
        else:
            dict_query[key] = value

    query_score = [(k, float(v)) for k, v in dict_query.items()]
    query_score.sort(key=lambda x:x[1], reverse=True)
    
    return query_score
    
    raise NotImplementedError()

In [24]:
docs_by_id = dict(docs)
def print_results(docs, len_limit=50):    
    for i, (doc_id, score) in enumerate(docs):
        doc_content = docs_by_id[doc_id].strip().replace("\n", "\\n")[:len_limit] + "..."
        print(f"Rank {i}({score:.2}): {doc_content}")

test_bow = bow_search("report", index_set=1)[:5]
print(f"BOW Results:")
print_results(test_bow)


BOW Results:
Rank 0(1.0): Preliminary Report-International Algebraic Languag...
Rank 1(1.0): ALGOL Sub-Committee Report - Extensions...
Rank 2(1.0): The Use of Computers in Engineering Classroom Inst...
Rank 3(1.0): Report on a Conference of University Computing Cen...
Rank 4(1.0): Report on the Algorithmic Language ALGOL 60...



### Section 3.2: TF-IDF <a class="anchor" id="tfidf"></a>

In [28]:
def compute_df(documents):
    """
        Compute the document frequency of all terms in the vocabulary
        Input: A list of documents
        Output: A dictionary with {token: document frequency)
    """
    
    doc_freq = {}
    
    for doc in documents:
        for word in set(doc):
            if word not in doc_freq.keys():
                doc_freq[word] = 1
            else:
                doc_freq[word] += 1
                
    return doc_freq
    
    raise NotImplementedError()


In [29]:
# get the document frequencies of each document
df_1 = compute_df([d[1] for d in doc_repr_1])
df_2 = compute_df([d[1] for d in doc_repr_2])

def get_df(index_set):
    assert index_set in {1, 2}
    return {
        1: df_1,
        2: df_2
    }[index_set]

In [31]:
def tfidf_search(query, index_set):
    """
        Perform a search over all documents with the given query using tf-idf. 
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    df = get_df(index_set)
    processed_query = preprocess_query(query, index_set)
        
    total_docs = len(docs)
    idf = {key: np.log(total_docs / value) for (key, value) in df.items()}

    query_score = []
    
    for query_word in processed_query:
        for key, value in index.items():
            if query_word == key:
                tfidf_value = [(doc_id, token_count * idf[key]) for doc_id, token_count in value]
                query_score.extend(tfidf_value)
    
    dict_query = dict()
    for (key, value) in query_score:
        if key in dict_query:
            dict_query[key] += value
        else:
            dict_query[key] = value
            
    query_score = [(k, float(v)) for k, v in dict_query.items()]
    query_score.sort(key=lambda x:x[1], reverse=True)
    
    return query_score
    
    raise NotImplementedError()

In [32]:
test_tfidf = tfidf_search("report", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf)

TFIDF Results:
Rank 0(2.3e+01): An Information Algebra - Phase I Report-Language\n...
Rank 1(2.3e+01): Rejuvenating Experimental Computer Science\nThis r...
Rank 2(1.2e+01): ALGOL 60 Confidential\nThe ALGOL 60 Report,* when ...
Rank 3(7.8): Automatic Abstracting and Indexing Survey and Reco...
Rank 4(7.8): A String Language for Symbol Manipulation Based on...


--- 

### Section 3.3: Query Likelihood Model <a class="anchor" id="qlm"></a>


Implement a QL model with Jelinek-Mercer Smoothing. That means an interpolated score is computed per word. In addition, it accumulates the scores by summing the **log** (smoothed) probability which leads to better numerical stability.

In [43]:
def ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a QL model 
        with Jelinek-Mercer Smoothing (set smoothing=0.1). 
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
        
    sm = 0.1
    rsm = 1 - 0.1
    
    coll_freq = {k: sum(count for doc_id, count in value) for k, value in index.items()}
    coll_length = sum(doc_lengths.values()) 
    
    query_score = []
    
    for query_word in processed_query:
        for key, value in index.items():
            if query_word == key:
                ql_value = [(doc_id,
                             np.log(rsm*(tkn_ct/doc_lengths[doc_id]) + sm*(coll_freq[key]/coll_length))) for doc_id, tkn_ct in value]
                query_score.extend(ql_value)
                
                doc_contains = [doc_id for doc_id, token_count in value]
                for i in range(len(docs)):
                    i += 1
                    if i not in doc_contains:
                        query_score.append((i, np.log(0 + sm*(coll_freq[key]/coll_length))))
    
    dict_query = dict()
    for (key, value) in query_score:
        if key in dict_query:
            dict_query[key] += value
        else:
            dict_query[key] = value

    query_score = [(k, float(v)) for k, v in dict_query.items()]
    query_score.sort(key=lambda x:x[1], reverse=True)
    
    return query_score
    
    raise NotImplementedError()
    

In [44]:
test_ql_results = ql_search("report", index_set=1)[:5]
print_results(test_ql_results)

Rank 0(-1.7): A Report Writer For COBOL...
Rank 1(-1.7): A CRT Report Generating System...
Rank 2(-1.9): Preliminary Report-International Algebraic Languag...
Rank 3(-1.9): Supplement to the ALGOL 60 Report...
Rank 4(-2.1): ALGOL Sub-Committee Report - Extensions...

Rank 0(-1.7e+01): A Report Writer For COBOL...
Rank 1(-1.7e+01): A CRT Report Generating System...
Rank 2(-1.9e+01): Preliminary Report-International Algebraic Languag...
Rank 3(-1.9e+01): Supplement to the ALGOL 60 Report...
Rank 4(-2.1e+01): ALGOL Sub-Committee Report - Extensions...


### Section 3.4: BM25 <a class="anchor" id="bm25"></a>

In [49]:
def bm25_search(query, index_set):
    """
        Perform a search over all documents with the given query using BM25. Use k_1 = 1.5 and b = 0.75
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    df = get_df(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
        
    k1 = 1.5
    b = 0.75
    k2 = k1 + 1
    b2 = 1 - b
    
    total_docs = len(docs)
    avg_dl = np.mean(list(doc_lengths.values()))
    idf = {key: np.log(total_docs / value) for (key, value) in df.items()}

    query_score = []
    
    for query_word in processed_query:
        for key, value in index.items():
            if query_word == key:
                bm_value = [(doc_id, idf[key] * ((k2*tkn_ct)/(k1*(b2+(b*doc_lengths[doc_id]/avg_dl))+tkn_ct))) for doc_id, tkn_ct in value]
                query_score.extend(bm_value)
    
    dict_query = dict()
    for (key, value) in query_score:
        if key in dict_query:
            dict_query[key] += value
        else:
            dict_query[key] = value
            
    query_score = [(k, float(v)) for k, v in dict_query.items()]
    query_score.sort(key=lambda x:x[1], reverse=True)
    
    return query_score
    
    raise NotImplementedError()

In [50]:
test_bm25_results = bm25_search("report", index_set=1)[:5]
print_results(test_bm25_results)


Rank 0(6.7): A Report Writer For COBOL...
Rank 1(6.7): A CRT Report Generating System...
Rank 2(6.6): Preliminary Report-International Algebraic Languag...
Rank 3(6.6): Supplement to the ALGOL 60 Report...
Rank 4(6.5): ALGOL Sub-Committee Report - Extensions...


In [55]:
# class for results
ResultRow = namedtuple("ResultRow", ["doc_id", "snippet", "score"])
# doc_id -> doc
docs_by_id = dict((d[0], d[1]) for d in docs)

def highlight_text(document, query, tol=17):
    import re
    tokens = tokenize(query)
    regex = "|".join(f"(\\b{t}\\b)" for t in tokens)
    regex = re.compile(regex, flags=re.IGNORECASE)
    output = ""
    i = 0
    for m in regex.finditer(document):
        start_idx = max(0, m.start() - tol)
        end_idx = min(len(document), m.end() + tol)
        output += "".join(["...",
                        document[start_idx:m.start()],
                        "<strong>",
                        document[m.start():m.end()],
                        "</strong>",
                        document[m.end():end_idx],
                        "..."])
    return output.replace("\n", " ")


def make_results(query, search_fn, index_set):
    results = []
    for doc_id, score in search_fn(query, index_set):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results


In [56]:
search_fn = bm25_search
index_set = 1

text = widgets.Text(description="Search Bar", width=200)
display(text)

def handle_submit(sender):
    print(f"Searching for: '{sender.value}'")
    
    results = make_results(sender.value, search_fn, index_set)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit)

Text(value='', description='Search Bar')

---

## Section 4: Evaluation <a class="anchor" id="evaluation"></a>

[Back to Part 1](#part1)

Use offline evaluation metrics to understand the performs of the algorithms.

In [58]:
def read_qrels(root_folder = "./datasets/"):   
    file_location = os.path.join(root_folder, "qrels.text")

    relevant_docs = {}
    
    with open(file_location, "r") as f:
        lines = f.readlines()
    
    qrels = []
    
    for line in lines:
        items_line = line.split(' ')
        qrels.append((int(items_line[0].lstrip('0')), int(items_line[1].lstrip('0'))))
        
    for x, y in qrels:
        relevant_docs.setdefault(x, []).append(y)
    
    return relevant_docs
    
    raise NotImplementedError()


In [60]:
def precision_k(results, relevant_docs, k):   
    return np.sum([1 for y in [x[0] for x in results[:k]] if y in relevant_docs])/k
    
    raise NotImplementedError()

In [61]:
qid = queries[0][0]
qtext = queries[0][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
precision = precision_k(results, qrels[qid], 10)
print(f'precision@10 = {precision}')


query: What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
precision@10 = 0.2


In [62]:
def recall_k(results, relevant_docs, k):
   
    to_return = np.sum([1 for y in [x[0] for x in  results[:k]] if y in relevant_docs])/len(relevant_docs)
    
    return to_return
    
    raise NotImplementedError()

In [63]:
qid = queries[10][0]
qtext = queries[10][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
recall = recall_k(results, qrels[qid], 10)
print(f'recall@10 = {recall}')


query: SETL, Very High Level Languages
recall@10 = 0.3157894736842105


In [64]:
def average_precision(results, relevant_docs):  
    position_k={}
    
    for i in range(1,len(results)+1,1):
        position_k[i]=precision_k(results, relevant_docs, i)
        
    index=[]
    for i,j in enumerate(results):
         if j[0] in relevant_docs:
            index.append(i+1)
            
    if len(index)>0:
        return sum([position_k[x] for x in index])/len(index)
    else:
        return 0
    
    raise NotImplementedError()


In [65]:
qid = queries[20][0]
qtext = queries[20][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
mean_ap = average_precision(results, qrels[qid])
print(f'MAP = {mean_ap}')


query: computational complexity, intractability, class-complete reductions,
algorithms and efficiency
MAP = 0.17269233631989397


In [66]:
def err(results, relevant_docs):  
    max_grade = 1.0
    step_down = 1.0
    error = 0.0

    for i, (doc_id, score) in enumerate(results):
        grade = 1. if doc_id in relevant_docs else 0.
        t = (pow(2, grade) - 1) / pow(2, max_grade)
        error += step_down * (t / (i + 1.))
        step_down *= (1. - t)
        
    return error
    
    raise NotImplementedError()

In [67]:
qid = queries[30][0]
qtext = queries[30][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
ERR = err(results, qrels[qid])
print(f'ERR = {ERR}')


query: I'd like to find articles describing the use of singular value decomposition
in digital image processing.  Applications include finding approximations
to the original image and restoring images that are subject to noise. An
article on the subject is H.C. Andrews and C.L. Patterson "Outer product
expansions and their uses in digital image processing", American Mathematical
Monthly, vol. 82.
ERR = 0.625



# Part 2: Semantic-based Matching <a class="anchor" id="part2"></a>

[Back to top](#top)

We will now experiment with methods that go beyond lexical methods like TF-IDF, which operate at the word level and are high dimensional and sparse, and look at methods which constructs low dimensional dense representations of queries and documents. 

Since these low-dimensional methods have a higher time complexity, they are typically used in conjunction with methods like BM-25. That is, instead of searching through potentially million documents to find matches using low dimensional vectors, a list of K documents are retrieved using BM25, and then **re-ranked** using the other method. 

LSI/LDA takes documents that are similar on a semantic level - for instance, if they are describing the same topic - and projects them into nearby vectors, despite having low lexical overlap.
Table of contents:
- [Section 6: LSI](#lsi) 
- [Section 7: LDA](#lda) 
- [Section 8: Word2Vec/Doc2Vec](#2vec) 
- [Section 8: Re-ranking](#reranking) 


## Section 6: Latent Semantic Indexing (LSI) <a class="anchor" id="lsi"></a>

[Back to Part 2](#part2)

LSI is one of the methods to embed the queries and documents into vectors. It is based on a method similar to Principal Component Analysis (PCA) for obtaining a dense concept matrix out of the sparse term-document matrix.

In [71]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import downloader as g_downloader
# gensim uses logging, so set it up 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [73]:
def dot(vec_1,vec_2): 
    vec_1 = list(list(zip(*vec_1))[1])
    vec_2 = list(list(zip(*vec_2))[1])
    dot_product = np.dot(vec_1,vec_2)
    return dot_product
    
    raise NotImplementedError()


def cosine_sim(vec_1, vec_2):
    
    cosine = dot(vec_1,vec_2)/(np.sqrt(dot(vec_1,vec_1))*np.sqrt(dot(vec_2,vec_2)))
    if np.isnan(cosine):
        return 0.0
    else:
        return cosine
    
    raise NotImplementedError()

In [76]:
class VectorSpaceRetrievalModel:
    """
        Parent class for Dense Vector Retrieval models
    """
    def __init__(self, doc_repr):
        """
            document_collection: 
                [
                    (doc_id_1, [token 1, token 2, ...]), 
                    (doc_id_2, [token 1, token 2, ....]) 
                    ...
                ]

        """
        self.doc_repr = doc_repr
        self.documents = [_[1] for _ in self.doc_repr]
        
        # construct a dictionary
        self.dictionary = Dictionary(self.documents)
        # Filter out words that occur less than 20 documents, or more than 50% of the documents.
        self.dictionary.filter_extremes(no_below=10)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.documents]
    
        # Make a index to word dictionary.
        temp = self.dictionary[0]  # This is only to "load" the dictionary.
        self.id2word = self.dictionary.id2token
        
        # this is set by the train_model function
        self.model = None
        
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        vectors = {}
        for (doc_id, _), cc in zip(self.doc_repr, self.corpus):
            vectors[doc_id] = self.model[cc]
        return vectors

    def vectorize_query(self, query):
        query = process_text(query, **config_2)
        query_vector = self.dictionary.doc2bow(query)
        return self.model[query_vector]
    
    def train_model(self):
        """
            Trains a model and sets the 'self.model' variable. 
            Make sure to use the variables created in the __init__ method.
            e.g the variables which may be useful: {corpus, dictionary, id2word}
        """
        raise NotImplementedError()

In [77]:
class LsiRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.num_topics = 100
        self.chunksize = 2000
    
    def train_model(self):        
        model = LsiModel(corpus=self.corpus, id2word=self.id2word, num_topics=self.num_topics, chunksize=self.chunksize)
        self.model = model
        return model
    
        raise NotImplementedError()

In [79]:
class DenseRetrievalRanker:
    def __init__(self, vsrm, similarity_fn):
        """
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.vsrm = vsrm 
        self.vectorized_documents = self.vsrm.vectorize_documents()
        self.similarity_fn = similarity_fn
    
    def _compute_sim(self, query_vector):
        """
            Compute the similarity of `query_vector` to documents in 
            `self.vectorized_documents` using `self.similarity_fn`
            Returns a list of (doc_id, score) tuples
        """        
        #list_scores = [(doc_id, self.similarity_fn(query_vector, vec)) for doc_id, vec in self.vectorized_documents.items()]
        list_scores = list()
        if not query_vector:
            return []
        for doc_id, vec in self.vectorized_documents.items():
            if not vec: #doc_id in [235, 913, 917, 2782]: I think we should do "if not vec:" instead
                score = 0
            else:
                score = self.similarity_fn(query_vector, vec)
            list_scores.append((doc_id, score))
            
        return list_scores
    
        raise NotImplementedError()
        
    
    def search(self, query):
        scores = self._compute_sim(self.vsrm.vectorize_query(query))
        scores.sort(key=lambda _:-_[1])
        return scores  

In [80]:
##### Function check
lsi = LsiRetrievalModel(doc_repr_2)
lsi.train_model()
drm_lsi = DenseRetrievalRanker(lsi, cosine_sim)
drm_lsi.search("report")[:5]
##### 

[(599, 0.7996616893459932),
 (947, 0.5727198744397531),
 (53, 0.5036188582993223),
 (1339, 0.4618035237113527),
 (3160, 0.44068710472917144)]

In [81]:
search_fn = drm_lsi.search

text = widgets.Text(description="Search Bar", width=200)
display(text)

def make_results_2(query, search_fn):
    results = []
    for doc_id, score in search_fn(query):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results

def handle_submit_2(sender):
    print(f"Searching for: '{sender.value}' (SEARCH FN: {search_fn})")
    
    results = make_results_2(sender.value, search_fn)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 7: Latent Dirichlet Allocation (LDA) <a class="anchor" id="lda"></a>

[Back to Part 2](#part2)

LDA, unlike LSI, outputs a topic **distribution**, not a vector. 


---

The Jenson-Shannon divergence is a symmetric and finite measure on two probability distributions (unlike the KL, which is neither). For identical distributions, the JSD is equal to 0, and since our code uses 0 as irrelevant and higher scores as relevant, we use `(1 - JSD)` as the score or 'similarity' in our setup

In [82]:
def jenson_shannon_divergence(vec_1, vec_2, assert_prob=False):
    
    if assert_prob:
        assert sum([v for k,v in vec_1]) == 1
        assert sum([v for k,v in vec_2]) == 1
        assert min([v for k,v in vec_1]) > 0
        assert min([v for k,v in vec_2]) > 0
    vec_1 = list(list(zip(*vec_1))[1])
    vec_2 = list(list(zip(*vec_2))[1])
    mean = list(0.5*(np.array(vec_1) + np.array(vec_2)))
    kl_divergence_1 = sum(vec_1[i]*np.log2(vec_1[i]/mean[i]) for i in range(len(vec_1)))
    kl_divergence_2 = sum(vec_2[i]*np.log2(vec_2[i]/mean[i]) for i in range(len(vec_2)))
    
    return 0.5*kl_divergence_1 + 0.5*kl_divergence_2
    
    raise NotImplementedError()

def jenson_shannon_sim(vec_1, vec_2, assert_prob=False):
    return 1 - jenson_shannon_divergence(vec_1, vec_2)




In [84]:
class LdaRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # use these parameters in the train_model method
        self.num_topics = 100
        self.chunksize = 2000
        self.passes = 20
        self.iterations = 400
        self.eval_every = 10
        # this is need to get full vectors
        self.minimum_probability=0.0
        self.alpha='auto'
        self.eta='auto'
    
    
    def train_model(self):
        
        model = LdaModel(corpus=self.corpus, id2word=self.id2word, num_topics=self.num_topics, chunksize=self.chunksize,\
                        passes=self.passes, alpha=self.alpha,eval_every=self.eval_every, minimum_probability=self.minimum_probability,\
                        eta=self.eta, iterations=self.iterations)
        self.model = model
        return model
    
        raise NotImplementedError()

In [86]:
lda = LdaRetrievalModel(doc_repr_2)
lda.train_model()
drm_lda = DenseRetrievalRanker(lda, jenson_shannon_sim)

search_fn = drm_lda.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

## Section 8: Word2Vec/Doc2Vec <a class="anchor" id="2vec"></a>

[Back to Part 2](#part2)

Word2Vec creates representations of words, not documents, so the word level vectors need to be aggregated to obtain a representation for the document. Here, we will simply take the mean of the vectors. 


In [87]:
class W2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # the dimensionality of the vectors
        self.size = 100 
        self.min_count = 1
    
    def train_model(self):
        """
        Trains the W2V model
        """
        
        model = Word2Vec(sentences=self.documents, size=self.size, min_count=self.min_count)
        self.model = model
        return model
    
        raise NotImplementedError()

    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        
        vectors = {}
        
        final_repr = list()
        for doc_id, tok in self.doc_repr:
            for word in tok:
                if word in self.model.wv.vocab:
                    vec = self.model.wv[word]
                    final_repr.append(vec)
                else:
                    final_repr.append(np.zeros((self.size,)))
            mean_vector = np.mean(np.array(final_repr),axis=0)
            vectors[doc_id] = list(enumerate(mean_vector))
        return vectors
    
        raise NotImplementedError()
            
    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        
        query = process_text(query, **config_2)
        final_repr = list()
        for word in query:
            if word in self.model.wv.vocab:
                vec = self.model.wv[word]
                
                final_repr.append(vec)
            else:
                final_repr.append(np.zeros((self.size,)))
        final_repr = np.mean(np.array(final_repr), axis=0)
        
    
        return list(enumerate(final_repr))
    
        raise NotImplementedError()
    
class W2VPretrainedRetrievalModel(W2VRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        self.model_name = "word2vec-google-news-300"
        self.size = 300
    
    def train_model(self):
        """
        Loads the pretrained model
        """
        self.model = g_downloader.load(self.model_name)

w2v = W2VRetrievalModel(doc_repr_2)
w2v.train_model()

# you can now get a W2V vector for a given query in the following way:
w2v.vectorize_query("report")

2021-02-18 09:46:14,573 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 09:46:14,702 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115978 corpus positions)
2021-02-18 09:46:14,708 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-18 09:46:14,709 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 09:46:14,712 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 09:46:14,775 : INFO : collecting all words and their counts
2021-02-18 09:46:14,777 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-18 09:46:14,796 : INFO : collected 5937 word types from a corpus of 115978 raw w

[(0, 0.086458154),
 (1, 0.47689313),
 (2, -0.116774),
 (3, -1.0261179),
 (4, 0.40345186),
 (5, -0.08808004),
 (6, -0.32522276),
 (7, 0.023351425),
 (8, -0.26708075),
 (9, -0.019330628),
 (10, -0.29249766),
 (11, 0.04949044),
 (12, -0.07726371),
 (13, -1.0515071),
 (14, 0.092795976),
 (15, -0.49762914),
 (16, 0.36850604),
 (17, -0.20609863),
 (18, 0.58328485),
 (19, -0.22554697),
 (20, 0.43124232),
 (21, -0.20167613),
 (22, -0.074251674),
 (23, 0.17088152),
 (24, -0.14901637),
 (25, 1.1308036),
 (26, -0.41290075),
 (27, -0.10039292),
 (28, 0.4723352),
 (29, 0.6130844),
 (30, -0.26171646),
 (31, 0.07677623),
 (32, 0.3208393),
 (33, -0.3194513),
 (34, -0.27150375),
 (35, 0.31206724),
 (36, -0.77079135),
 (37, -0.7586395),
 (38, 0.228283),
 (39, 1.1315128),
 (40, 0.051851362),
 (41, 0.010796506),
 (42, 0.3717191),
 (43, -0.75109404),
 (44, 0.07001864),
 (45, -0.66212624),
 (46, 0.36532757),
 (47, -0.39243025),
 (48, 0.72810745),
 (49, -0.2040465),
 (50, 0.366937),
 (51, 0.07305568),
 (52, 

In [89]:
w2v_pretrained = W2VPretrainedRetrievalModel(doc_repr_2)
w2v_pretrained.train_model()

# you can now get an W2V vector for a given query in the following way:
w2v_pretrained.vectorize_query("report")

2021-02-18 09:46:16,347 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 09:46:16,494 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115978 corpus positions)
2021-02-18 09:46:16,502 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-18 09:46:16,503 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 09:46:16,506 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 09:46:16,862 : INFO : loading projection weights from C:\Users\marta/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2021-02-18 09:46:57,830 : INFO : loaded (3000000, 300) matrix from C:\Users\marta/gensim-data\word2vec-google-new

[(0, -0.14257812),
 (1, -0.1640625),
 (2, -0.09033203),
 (3, -0.11230469),
 (4, 0.100097656),
 (5, -0.041259766),
 (6, 0.048828125),
 (7, -0.13671875),
 (8, 0.19628906),
 (9, -0.13476562),
 (10, -0.017578125),
 (11, 0.032226562),
 (12, 0.095214844),
 (13, -0.10595703),
 (14, -0.16992188),
 (15, 0.041015625),
 (16, -0.26367188),
 (17, -0.0063171387),
 (18, -0.17773438),
 (19, -0.24023438),
 (20, 0.3515625),
 (21, -0.012207031),
 (22, -0.16210938),
 (23, -0.12060547),
 (24, 0.04321289),
 (25, 0.10986328),
 (26, 0.052490234),
 (27, 0.17871094),
 (28, -0.14550781),
 (29, 0.13769531),
 (30, -0.08203125),
 (31, -0.28320312),
 (32, -0.10888672),
 (33, -0.2890625),
 (34, 0.072265625),
 (35, -0.04736328),
 (36, 0.040283203),
 (37, 0.067871094),
 (38, 0.11669922),
 (39, 0.000831604),
 (40, 0.068359375),
 (41, 0.12011719),
 (42, -0.088378906),
 (43, 0.33789062),
 (44, -0.044677734),
 (45, -0.030151367),
 (46, 0.0076904297),
 (47, -0.021118164),
 (48, -0.25390625),
 (49, 0.14941406),
 (50, 0.39843

In [91]:
drm_w2v = DenseRetrievalRanker(w2v, cosine_sim)

# test your LDA model
search_fn = drm_w2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

In [92]:
drm_w2v_pretrained = DenseRetrievalRanker(w2v_pretrained, cosine_sim)

# test your LDA model
search_fn = drm_w2v_pretrained.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

<ipython-input-87-9aa5b93747f8>:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in self.model.wv.vocab:
<ipython-input-87-9aa5b93747f8>:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec = self.model.wv[word]


Text(value='', description='Search Bar')

In [93]:
class D2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.vector_size= 100
        self.min_count = 1
        self.epochs = 20
        
        
        self.tags = [TaggedDocument(doc, [i]) for i, doc in self.doc_repr]
        self.model = None
        
        
    def train_model(self):
        
        model = Doc2Vec(self.tags, vector_size=self.vector_size, min_count=self.min_count, epochs=self.epochs)
        self.model = model
        return model
    
        raise NotImplementedError()
    
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        
        vectors = {}
        for doc_id, tok in self.doc_repr:
            vectors[doc_id] = list(enumerate(self.model.infer_vector(tok)))
        return vectors
        
        raise NotImplementedError()

    def vectorize_query(self, query):
        # YOUR CODE HERE
        
        query = process_text(query, **config_2)
        return list(enumerate(self.model.infer_vector(query)))
    
        raise NotImplementedError()
        
d2v = D2VRetrievalModel(doc_repr_2)
d2v.train_model()


# # you can now get an LSI vector for a given query in the following way:
d2v.vectorize_query("report")

2021-02-18 09:51:12,577 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 09:51:12,716 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115978 corpus positions)
2021-02-18 09:51:12,723 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-18 09:51:12,724 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 09:51:12,727 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 09:51:12,802 : INFO : collecting all words and their counts
2021-02-18 09:51:12,802 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-02-18 09:51:12,827 : INFO : collected 5937 word types and 3205 unique tags fro

2021-02-18 09:51:18,083 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-18 09:51:18,083 : INFO : EPOCH - 16 : training on 115978 raw words (95650 effective words) took 0.2s, 433495 effective words/s
2021-02-18 09:51:18,283 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-18 09:51:18,294 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-18 09:51:18,301 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-18 09:51:18,302 : INFO : EPOCH - 17 : training on 115978 raw words (95576 effective words) took 0.2s, 448061 effective words/s
2021-02-18 09:51:18,502 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-18 09:51:18,521 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-18 09:51:18,529 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-18 09:51:18,529 : INFO : EPOCH - 18 : training on 115978 raw words (95529 effective w

[(0, 0.039129663),
 (1, 0.06380455),
 (2, 0.0372417),
 (3, -0.066919826),
 (4, 0.029172387),
 (5, 0.043593843),
 (6, -0.07972468),
 (7, -0.006866838),
 (8, -0.029241165),
 (9, 0.04472167),
 (10, 0.007805907),
 (11, -0.02276804),
 (12, -0.012108639),
 (13, -0.0821464),
 (14, 0.06447723),
 (15, -0.047652494),
 (16, 0.067281485),
 (17, -0.018826548),
 (18, 0.09013158),
 (19, -0.038388304),
 (20, 0.04685946),
 (21, -0.03749524),
 (22, -0.050237402),
 (23, 0.043843053),
 (24, -0.018520087),
 (25, 0.15471224),
 (26, 0.05769825),
 (27, -0.036037397),
 (28, 0.06086186),
 (29, 0.08397715),
 (30, -0.08648466),
 (31, 0.058906913),
 (32, 0.06772328),
 (33, -0.011261158),
 (34, -0.009952199),
 (35, -0.0033186167),
 (36, -0.056504942),
 (37, -0.054663632),
 (38, 0.042858183),
 (39, 0.10614691),
 (40, 0.030201562),
 (41, -0.025412027),
 (42, -0.006855831),
 (43, -0.06881432),
 (44, 0.015810946),
 (45, -0.054090057),
 (46, 0.088405214),
 (47, -0.03807256),
 (48, 0.13304307),
 (49, -0.035028167),
 (50,

In [95]:
drm_d2v = DenseRetrievalRanker(d2v, cosine_sim)

# test your LDA model
search_fn = drm_d2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 9: Re-ranking <a class="anchor" id="reranking"></a>

[Back to Part 2](#part2)

To motivate the re-ranking perspective (i.e retrieve with lexical method + rerank with a semantic method), let's search using semantic methods and compare it to BM25's performance, along with their runtime:


---

Re-ranking involves retrieving a small set of documents using simple but fast methods like BM25 and then re-ranking them with the aid of semantic methods such as LDA or LSI. This not only makes retrieval faster, but semantic methods perform poorly when used in isolation.

In [97]:
class DenseRerankingModel:
    def __init__(self, initial_retrieval_fn, vsrm, similarity_fn):
        self.ret = initial_retrieval_fn
        self.vsrm = vsrm
        self.similarity_fn = similarity_fn
        self.vectorized_documents = vsrm.vectorize_documents()
        
        assert len(self.vectorized_documents) == len(doc_repr_2)
    
    def search(self, query, K=50):
        
        initial_list = self.ret(query)[:K]
        query = self.vsrm.vectorize_query(query)
        list_scores = list()
        for doc_id, _ in initial_list:
            doc_vec = self.vectorized_documents[doc_id]
            score = self.similarity_fn(query, doc_vec)
            list_scores.append((doc_id, score))
        list_scores.sort(key=lambda _:-_[1])
        return list_scores
    
        raise NotImplementedError()